In [1]:
import numpy as np 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit import Aer
from qiskit.primitives import Sampler, BackendSampler
from qiskit.circuit.library import TwoLocal

from src.Quantum_Optimizer_COBYLA import Quantum_Solver

In [2]:
target_mdl = QuadraticProgram()
target_mdl.read_from_lp_file('instance/seq2.lp')

N = target_mdl.get_num_vars()
print(N)

#print(target_mdl.prettyprint())

22


In [3]:
cplex_result = CplexOptimizer().solve(target_mdl)
cplex_optval = cplex_result.fval
cplex_xsol = cplex_result.x.astype(int)

print(
        '-'*110,
        '[CPLEX Result]',
        f'   optimal value: {cplex_optval:.3f}',
        f'   optimal solution (binary): {cplex_xsol}',
        f'   status: {cplex_result.status.name}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: -42.100
   optimal solution (binary): [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0]
   status: SUCCESS
--------------------------------------------------------------------------------------------------------------


In [4]:
qubo = QuadraticProgramToQubo().convert(target_mdl)

#print(qubo.prettyprint())

In [5]:
num_qubits = N
print(f'number of system qubits: {num_qubits}')

number of system qubits: 22


In [6]:
reps = 2

ansatz = TwoLocal(
    num_qubits = num_qubits,
    rotation_blocks = 'ry', 
    entanglement_blocks = 'cz',
    entanglement = 'pairwise', 
    reps = reps,
    insert_barriers = True
    )

n_params = ansatz.num_parameters
print(f'number of params: {n_params}')

number of params: 66


In [7]:
shots = 2**10
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 1024


In [8]:
def objective(x):
    objval = qubo.objective.evaluate(x)
    return objval

In [9]:
QuantumOptimizer = Quantum_Solver()
QuantumOptimizer.set_optimizer(num_qubits, ansatz, sampler, objective)

theta_initial = np.hstack([np.full(num_qubits, np.pi/2), np.zeros(num_qubits*ansatz.reps)])
result = QuantumOptimizer.run(qc_params_init=theta_initial, maxiter=300)

step = 0 	 fval = 301.500
step = 1 	 fval = 282.700
step = 2 	 fval = 282.700
step = 3 	 fval = 282.700
step = 4 	 fval = 282.700
step = 5 	 fval = 282.700
step = 6 	 fval = 282.700
step = 7 	 fval = 282.700
step = 8 	 fval = 282.700
step = 9 	 fval = 282.700
step = 10 	 fval = 282.700
step = 11 	 fval = 282.700
step = 12 	 fval = 282.700
step = 13 	 fval = 282.700
step = 14 	 fval = 282.700
step = 15 	 fval = 282.700
step = 16 	 fval = 282.700
step = 17 	 fval = 282.700
step = 18 	 fval = 282.700
step = 19 	 fval = 282.700
step = 20 	 fval = 282.700
step = 21 	 fval = 282.700
step = 22 	 fval = 282.700
step = 23 	 fval = 282.700
step = 24 	 fval = 282.700
step = 25 	 fval = 282.700
step = 26 	 fval = 282.700
step = 27 	 fval = 282.700
step = 28 	 fval = 282.700
step = 29 	 fval = 282.700
step = 30 	 fval = 282.700
step = 31 	 fval = 282.700
step = 32 	 fval = 282.700
step = 33 	 fval = 282.700
step = 34 	 fval = 282.700
step = 35 	 fval = 282.700
step = 36 	 fval = 282.700
step = 37 	

In [10]:
x = QuantumOptimizer.x
objval = qubo.objective.evaluate(x)

print(
        '-'*110,
        '[Quantum Result]',
        f'   objective:   {objval:.3f}',
        f'   solution:    {x}',
        f'   is feasible: {target_mdl.is_feasible(x)}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[Quantum Result]
   objective:   -42.100
   solution:    [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0]
   is feasible: True
--------------------------------------------------------------------------------------------------------------
